# The Overlap Learner
This brief piece of documentation is intended to present the Overlap Learner framework more formally, linking to the work of [Li et. al. (2014)](https://arxiv.org/pdf/1404.1785.pdf). [Li et. al. (2014)](https://arxiv.org/pdf/1404.1785.pdf) propose a unified framework for balancing weights, and a new weighting scheme called the "overlap weights", to carry out causal inference in presence of observational data. 

## The Setup
Consider a binary treatment $Z_i \in \{0, 1\}$, a continuous outcome $Y_i \in \mathbb{R}$ (or binary as well), and a set of features $\mathbf{X}_i \in \mathcal{X}$. Propensity score, defined as $\pi(x_i) = \mathbb{P} (Z_i = 1 | X_i = x_i) $, is very often use in observational studies to recover balance in the two treatment groups and mimic as much as possible a randomized experiment setting. A usual approach is to weight the outcome $Y_i$ by the Inverse Propensity Score Weights (IPW), defined as:

\begin{equation}
  w_z(x_i)=
  \begin{cases}
    \frac{1}{\pi(x_i)} & \text{if } Z_i=1 \\
    \frac{1}{1 - \pi(x_i)} & \text{if } Z_i=0
  \end{cases}
\end{equation}

IPW scheme has several advantages, but runs into severe issues of "exploding" bias and variance when $\pi(x_i)$ assumes extreme values (close to 0 or 1) and covariates distribution is unbalanced between the treated and control groups. In these cases, overlap assumption $0 < \pi(x_i) <1$ is threatened. [Li et. al. (2014)](https://arxiv.org/pdf/1404.1785.pdf) tackle issues of non-overlap scenarios by proposing a new set of "overlap weights" defined as:

\begin{equation}
  w_z(x_i)=
  \begin{cases}
    1 - \pi(x_i) & \text{if } Z_i=1 \\
    \pi(x_i) & \text{if } Z_i=0
  \end{cases}
\end{equation}

The rationale behind "overlap weights" is that they attempt to recreate balance in the two treated groups by weighting each units by their probability of being assigned to the opposite treatment group, and by doing this they assigns more emphasis on the population closest to a randomized experiment (i.e. on units that could have been seen in either group with similar probability - overlapping).

## The O-Learner
The O-Learner combines the idea of overlap weights and doubly-robust estimation to develop a "Meta-Learner" algorithm for the estimation of Individual/Heterogeneous Treatment Effects (ITE) when dealing with observational studies, where overlap assumption is often violated. Doubly-robust estimation finds its foundation in [Cassell et. al. (1976)](https://www.jstor.org/stable/2335742?seq=1) and [Robins et. al.](https://www.jstor.org/stable/2290910?seq=1), and has been later extensively used in policy learning and treatment effects estimation [Dudik et. al. (2011)](https://icml.cc/2011/papers/554_icmlpaper.pdf), [Bang et. al. (2005)](https://onlinelibrary.wiley.com/doi/pdf/10.1111/j.1541-0420.2005.00377.x), [Kennedy (2020)](https://arxiv.org/pdf/2004.14497.pdf) (and many others). The O-Learner can make use of more or less any base machine learning regression/classification algorithm (linear regression, tree ensembles, neural nets, etc.) found in the `sklearn` library.

O-Learner fits Conditional Average Treatment Effect (CATE), defined as $\tau (x_i) = \mathbb{E} [Y^{(1)} - Y^{(0)} | X_i = x_i]$, where $Y^{(Z_i)}$ is the potential outcome for the realization of $Z_i$, in four steps:

1. The first step estimates a baseline regression model $Y_i = g(X_i, Z_i) + \varepsilon_i$. This can be implemented as a S-Learner (`OLearner` class in `olearner.py`), i.e. learn a single surface $g_(X_i, Z_i)$, or as a T-Learner (`OTLearner` class in `olearner.py`), i.e. divide the sample into two subsample based on $Z_i$ and learn $g_1 (X_i)$ and $g_0 (X_i)$. See [Caron et. al. (2020)](https://arxiv.org/pdf/2009.06472.pdf) for more details on S, T and other type of Meta-Learners and their properties.

2. The second step consists in fitting a probabilistic classifier to get estimates of the PS $\pi(x_i)$ (regressing $Z_i$ on $X_i$, or on a different subset of covariates $W_i$).

3. The third step constructs the overlap-weighted doubly-robust outcomes as: 
\begin{equation}
 Y^{\mathcal{O}}_{i, z_i} = g_z (X_i) + \big( Y_i - g_z (X_i) \big) \cdot \mathbb{P} (Z_i = 1 - z_i | X_i) \cdot I (Z_i = z_i)
\end{equation}
4. Fit a final model for CATE and regress $Y^{\mathcal{O}}_{i, 1} - Y^{\mathcal{O}}_{i, 1}$ on $X_i$.


## Simulated Example
We illustrate the method's use on simulated data, with the test file `olearner\test_model.py`, in comparison to other Meta-Learners, namely S, T, X, DR Learners (found in the `econml` python library). 

In this example, we use simple linear and logistic regressions as base-learners (both in O-Learner and the other methods), but note that almost any `sklearn` Regression/Classification method is suitable and can be specified in the O-Learner class arguments (this holds for other meta-learners too). 

`olearner\olearner.py` has two type of importable classes:
- `OLearner`: O-Learner where $g(\cdot)$ baseline model is estimated as S-Learner
- `OTLearner`: O-Learner where $g(\cdot)$ baseline model is estimated as T-Learner

We set $N=1000$ and run the simulated study for $B=200$ iterations. Specifically, we use $\sqrt{\text{ PEHE}}$, a type of squared loss function on CATE (see [Caron et. al. (2020)](https://arxiv.org/pdf/2009.06472.pdf) among others for explanation), averaged over the $B$ simulations, to evaluate models performance. 

The averaged PEHE for each model on the simulated data is:

In [1]:
import olearner.test_model

      S-Learner  T-Learner  X-Learner  DR-Learner  O-Learner  OT-Learner
PEHE   0.451997   0.976102   0.976102    0.533272   0.402727    0.894139
